In [1]:
%cd ..

/home/erik/GitHub/melp


In [2]:
#import ROOT
import numpy as np
import matplotlib.pyplot as plt
from melp import Detector
import melp
import melp.taft

Welcome to JupyROOT 6.24/06


In [3]:
#import melp.legacy as melp

#test = melp.TileHitAngle("run42_20000.root", "outtest_1")
#test = melp.TileHitAngle("mu3e_run_050000.root", "outtest_big")

In [4]:
#mu3e_detector = dt.Detector(test.tile_id_pos, test.tile_id_pos, [0,1,2])

In [5]:
#mu3e_detector = Detector.initFromROOT("run42_20000.root")
#mu3e_detector = Detector.initFromROOT("sorted_truth.root")
#mu3e_detector = Detector.initFromSave("test_detector_file")
mu3e_detector = Detector.initFromROOT("./testdata/sorted/sorted1_big.root")

------------------------------
Detector information

Stats:
  - Tiles:  5824
    -> misal:  False
  - Pixel Modules:  2844
  - Loaded Runs (Tiles):  []
  - Loaded Runs (Pixel):  []
------------------------------


In [6]:
#mu3e_detector.save("test_detector_file")

In [7]:
melp.select(mu3e_detector)

In [8]:
melp.info()

Detector(TileDetector=Loaded Tiles: 5824, SensorModules=<melp.src.sensor.SensorModule object at 0x7f28aa8c7970>))


In [9]:
print(mu3e_detector)

Detector(TileDetector=Loaded Tiles: 5824, SensorModules=<melp.src.sensor.SensorModule object at 0x7f28aa8c7970>))


In [10]:
melp.addTileHits("./testdata/sorted/sorted1_big.root", truth=True, traj=False)
#melp.addTileHits("run42_20000.root", truth=True, traj=False)
#melp.addTileHits("mu3e_run_000133_sorted_truth.root", truth=True, traj=False)
#melp.addTileHits("mu3e_run_000134_sorted_truth.root", truth=True, traj=False)

AttributeError: 'TTree' object has no attribute 'p_in_x'

In [ ]:
mu3e_detector.TileDetector.tile[200000].hits

In [ ]:
mu3e_detector.TileDetector.tile[200000].info()

In [ ]:
mu3e_detector.info()

In [ ]:
rate_data = melp.getHitRate()

In [ ]:
rate_data_np = np.array(rate_data)

In [ ]:
result_rate = np.zeros((52*2,6))

for z_pos in range(2*52):
    result_rate[z_pos][0] = rate_data_np[0][z_pos*56+1]
    for phi_pos in range(56):
        result_rate[z_pos][1] += rate_data_np[1][z_pos*56+1+phi_pos] # total
        result_rate[z_pos][2] += rate_data_np[2][z_pos*56+1+phi_pos]
        result_rate[z_pos][3] += rate_data_np[3][z_pos*56+1+phi_pos]
        result_rate[z_pos][4] += rate_data_np[4][z_pos*56+1+phi_pos]
        result_rate[z_pos][5] += rate_data_np[5][z_pos*56+1+phi_pos] # edep
        

In [ ]:
# Umrechnung Rate -> pro sekunde
number_of_frames = 20000*3
frame_length = 64 #ns
ns_to_s = 1e-9

run_time_length = (number_of_frames*frame_length*ns_to_s) # s

number_of_tiles_phi = 56

fac = 1/(run_time_length*number_of_tiles_phi)
fac_khz = fac/1000 


In [ ]:
#import seaborn as sns


In [ ]:
#plt.bar(result_rate.T[0], result_rate.T[1], width = 2)
#sns.set_theme(style="darkgrid")

#fig = plt.figure(figsize=(10, 8))

#sns.histplot(x=result_rate.T[0], weights=result_rate.T[1]*fac_khz, bins=220, label="total rate")
#sns.histplot(x=result_rate.T[0], weights=result_rate.T[2]*fac_khz, bins=220, label="primary rate")
#sns.histplot(x=result_rate.T[0], weights=result_rate.T[3]*fac_khz, bins=220, label="secondary rate")
#sns.histplot(x=result_rate.T[0], weights=result_rate.T[4]*fac_khz, bins=220, label="tertiary rate")
#plt.legend(fontsize=15)
#plt.yticks([])
#plt.ylabel("Rate [khz]")
#plt.xlabel("z - position")
#plt.show()

In [ ]:
import matplotlib as mpl
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=["264653","e76f51","e9c46a","2a9d8f"]) 
# https://coolors.co/palettes/popular
plt.rcParams.update({'font.size': 20})


In [ ]:
#plt.bar(result_rate.T[0], result_rate.T[1], width = 2)
fig, ax = plt.subplots(figsize=(10, 8))

ax.hist(result_rate.T[0], weights=result_rate.T[1]*fac_khz, bins=220, range=(min(result_rate.T[0]),max(result_rate.T[0])), label="total rate")
#plt.hist(result_rate.T[0], weights=result_rate.T[2]*fac_khz, bins=220, range=(min(result_rate.T[0]),max(result_rate.T[0])), label="primary rate", stacked=True)
#plt.hist(result_rate.T[0], weights=result_rate.T[3]*fac_khz, bins=220, range=(min(result_rate.T[0]),max(result_rate.T[0])), label="secondary rate", stacked=True)
ax.hist([result_rate.T[0],result_rate.T[0],result_rate.T[0]], weights=[result_rate.T[2]*fac_khz, result_rate.T[3]*fac_khz, result_rate.T[4]*fac_khz], bins=220, range=(min(result_rate.T[0]),max(result_rate.T[0])), label=["primary rate","secondary rate","tertiary rate"], stacked=True)
ax.legend(fontsize=15)
ax.set_ylabel("Rate [khz]")
ax.set_xlabel("z - position")

ax2 = ax.twinx()
mn, mx = ax.get_ylim()
ax2.set_ylim(mn*1, (1-np.exp(-mx*1000*150*10**-9))*100)
ax2.set_ylabel("Pile-up [%]")

plt.show()

In [ ]:
rate_total_1 = sum((result_rate.T[2]*fac_khz)[:52])/52
rate_total_2 = sum((result_rate.T[2]*fac_khz)[52:])/52

print("Upstream Rate:", rate_total_1)
print("Downstream Rate:", rate_total_2)
print("Max rate:", max(result_rate.T[2]*fac_khz))
print("Min rate:", min(result_rate.T[2]*fac_khz))

In [ ]:
rate_total_1 = sum((result_rate.T[1]*fac_khz)[:52])/52
rate_total_2 = sum((result_rate.T[1]*fac_khz)[52:])/52

print("Upstream Rate:", rate_total_1)
print("Downstream Rate:", rate_total_2)
print("Max rate:", max(result_rate.T[1]*fac_khz))
print("Min rate:", min(result_rate.T[1]*fac_khz))

In [ ]:
#plt.bar(result_rate.T[0], result_rate.T[1], width = 2)
fig = plt.figure(figsize=(10, 8))
plt.hist(result_rate.T[0], weights=result_rate.T[5]*fac/1000, bins=220, range=(min(result_rate.T[0]),max(result_rate.T[0])), label="energy deposition")
plt.legend(fontsize=15)
#plt.yticks([])
plt.ylabel("Energy deposition [GeV/s]")
plt.xlabel("z - position")
plt.show()

In [ ]:
edep_hits = []
for tile in mu3e_detector.TileDetector.tile:

    for hit in mu3e_detector.TileDetector.tile[tile].hits:

        edep = hit.edep
        edep_hits.append(edep)

In [ ]:
fig = plt.figure(figsize=(10, 8))
plt.hist(edep_hits,bins=150, range=(0,3))
plt.show()
print(min(edep_hits))

In [ ]:
#melp.addSensorHits("run42_20000.root", traj=True)

In [ ]:
hitangle = melp.getHitAngle(rec_type="Truth", angle="theta", hit_type="primary")

In [ ]:
#print(hitangle[1])

In [ ]:
mu3e_detector.info()

In [ ]:
#mu3e_detector.SensorsModules.sensor[1].hit

In [ ]:
#mu3e_detector.TileDetector.tile[200000].hits

In [ ]:
binned_data, xedges, yedges = np.histogram2d(hitangle[0], hitangle[1], bins=[220, 180])

In [ ]:
plt.rcParams.update({'font.size': 20})


fig = plt.figure(figsize=(10, 8))

ax = fig.add_subplot(111, title='theta distribution')
X, Y = np.meshgrid(xedges, yedges)
im = ax.pcolormesh(X, Y, binned_data.T, cmap="viridis", norm = mpl.colors.LogNorm())
plt.ylabel("theta [deg]")
plt.xlabel("z [mm]")
cbar = plt.colorbar(im)
cbar.set_label('A.U.', rotation=270)
plt.show()

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def fit_func(x, a, b):
    return a*x + b

In [ ]:
pos = []
station_test = 200000

for z in range(len(mu3e_detector.TileDetector.row_ids(0, station_test))):
    z_pos = mu3e_detector.TileDetector.tile[mu3e_detector.TileDetector.column_ids(z, station_test)[0]].pos
    pos.append(z_pos[2])

In [ ]:
average_angle = []

for i in range(0,110):
    if sum(binned_data[i]) != 0:
        average_angle.append(np.average(np.array(range(0,180,1)),weights=binned_data[i]))
        
popt, cov = curve_fit(fit_func, pos[1:-2], average_angle[1:-2])
plt.plot(pos, average_angle, marker=".")
plt.plot(pos, fit_func(np.array(pos), popt[0],popt[1]))
plt.show()
print("Average Angle Upstream:", np.median(average_angle))
print("Angle Fit Params: ", popt)
len(average_angle)

In [ ]:
pos = []
station_test = 300000

for z in range(len(mu3e_detector.TileDetector.row_ids(0, station_test))):
    z_pos = mu3e_detector.TileDetector.tile[mu3e_detector.TileDetector.column_ids(z, station_test)[0]].pos
    pos.append(z_pos[2])

In [ ]:
average_angle = []

for i in range(110,220):
    if sum(binned_data[i]) != 0:
        average_angle.append(np.average(np.array(range(0,180,1)),weights=binned_data[i]))
        
popt, cov = curve_fit(fit_func, pos[1:-2], average_angle[2:-2])
plt.plot(pos, average_angle[1:], marker=".")
plt.plot(pos, fit_func(np.array(pos), popt[0],popt[1]))
plt.show()
print("Average Angle Downstream:", np.median(average_angle))
print("Angle Fit Params: ", popt)
len(average_angle)

In [ ]:
hitangle = melp.getHitAngle(rec_type="Truth", angle="theta", hit_type="all")
binned_data, xedges, yedges = np.histogram2d(hitangle[0], hitangle[1], bins=[220, 180])

In [ ]:
plt.rcParams.update({'font.size': 20})


fig = plt.figure(figsize=(10, 8))

ax = fig.add_subplot(111, title='theta distribution')
X, Y = np.meshgrid(xedges, yedges)
im = ax.pcolormesh(X, Y, binned_data.T, cmap="magma_r", norm = mpl.colors.LogNorm())
plt.ylabel("theta [deg]")
plt.xlabel("z [mm]")
cbar = plt.colorbar(im)
cbar.set_label('A.U.', rotation=270)
plt.show()

In [ ]:
hitangle = melp.getHitAngle(rec_type="Truth", angle="phi", hit_type="primary")
binned_data, xedges, yedges = np.histogram2d(hitangle[0], hitangle[1], bins=[220, 180])

In [ ]:
plt.rcParams.update({'font.size': 20})


fig = plt.figure(figsize=(10, 8))

ax = fig.add_subplot(111, title='phi distribution')
X, Y = np.meshgrid(xedges, yedges)
im = ax.pcolormesh(X, Y, binned_data.T, cmap="viridis", norm = mpl.colors.LogNorm())
plt.ylabel("phi [deg]")
plt.xlabel("z [mm]")
cbar = plt.colorbar(im)
cbar.set_label('A.U.', rotation=270)
plt.show()

In [ ]:
average_angle = []

for i in range(0,110):
    if sum(binned_data[i]) != 0:
        average_angle.append(np.average(np.array(range(-180,180,2)),weights=binned_data[i]))
plt.plot(average_angle, marker=".")
plt.show()
print("Average Angle Upstream:", np.median(average_angle))

In [ ]:
average_angle = []

for i in range(110,220):
    if sum(binned_data[i]) != 0:
        average_angle.append(np.average(np.array(range(-180,180,2)),weights=binned_data[i]))
plt.plot(average_angle, marker=".")
plt.show()
print("Average Angle Upstream:", np.median(average_angle))

In [ ]:
hitangle = melp.getHitAngle(rec_type="Truth", angle="phi", hit_type="all")
binned_data, xedges, yedges = np.histogram2d(hitangle[0], hitangle[1], bins=[220, 180])

In [ ]:
plt.rcParams.update({'font.size': 20})


fig = plt.figure(figsize=(10, 8))

ax = fig.add_subplot(111, title='phi distribution')
X, Y = np.meshgrid(xedges, yedges)
im = ax.pcolormesh(X, Y, binned_data.T, cmap="magma_r", norm = mpl.colors.LogNorm())
plt.ylabel("phi [deg]")
plt.xlabel("z [mm]")
cbar = plt.colorbar(im)
cbar.set_label('A.U.', rotation=270)
plt.show()

In [ ]:
hitangle = melp.getHitAngle(rec_type="Truth", angle="norm", hit_type="primary")
binned_data, xedges, yedges = np.histogram2d(hitangle[0], hitangle[1], bins=[220, 180])

In [ ]:
plt.rcParams.update({'font.size': 20})


fig = plt.figure(figsize=(10, 8))

ax = fig.add_subplot(111, title='norm distribution')
X, Y = np.meshgrid(xedges, yedges)
im = ax.pcolormesh(X, Y, binned_data.T, cmap="magma_r", norm = mpl.colors.LogNorm())
plt.ylabel("norm [deg]")
plt.xlabel("z [mm]")
cbar = plt.colorbar(im)
cbar.set_label('A.U.', rotation=270)
plt.show()

In [ ]:
average_angle = []

for i in range(0,110):
    if sum(binned_data[i]) != 0:
        average_angle.append(np.average(np.array(range(0,180,1)),weights=binned_data[i]))
plt.plot(average_angle, marker=".")
plt.show()
print(-np.median(average_angle))

In [ ]:
average_angle = []

for i in range(110,220):
    if sum(binned_data[i]) != 0:
        average_angle.append(np.average(np.array(range(0,180,1)),weights=binned_data[i]))
plt.plot(average_angle, marker=".")
plt.show()
print(-np.median(average_angle))

In [ ]:
hid = []

for tileid in mu3e_detector.TileDetector.tile:
    for hit in mu3e_detector.TileDetector.tile[tileid].hits:
        hid.append(hit.hid)

In [ ]:
hid_hist, bin_edges = np.histogram(hid, bins=14, range=(0,15))

norm = hid_hist[0]
print(norm)
for i in range(0,14):
    hid_hist[i] = hid_hist[i] / norm *100
    pass

In [ ]:
plt.rcParams.update({'font.size': 20})


fig = plt.figure(figsize=(10, 7))

plt.bar(range(1,15),hid_hist)
plt.yticks(np.arange(0, 101, step=25), ['0%','25%', '50%', '75%', '100%'])  # Set label locations.
plt.xticks(np.arange(1,15, step = 2))
plt.xlabel("number of tile hits")
plt.title("Percentage of tracks with the specified number of tile hits or more")
plt.show()

In [ ]:
x = 0
for i in hid:
    if i == 1:
        x+= 1
x

In [ ]:
hid_hist[0]

In [ ]:
hid_new = np.delete(hid, np.where(np.asarray(hid) == 1))
np.mean(hid_new)

In [ ]:
np.average(range(2,14), weights=hid_hist[1:-1])

In [ ]:
np.mean(hid)